In [31]:
import pandas as pd
import re
import os

##### Use one data folder as an example

In [32]:
bias = 'control'

In [33]:
# read and merge[]
data_folder = "/Users/claregrogan/Documents/GitHub/msc_bias_llm_project/emotion_experiments/results/persona_emotion_{}".format(bias)
files = os.listdir(data_folder)
folders = [item for item in files if os.path.isdir(os.path.join(data_folder, item))]
df_list = []

for folder in folders:
    folder_path = os.path.join(data_folder, folder)
    files = os.listdir(folder_path)
    csv_files = [f for f in files if f.endswith('.csv')]
    
    for file in csv_files:
        df = pd.read_csv(os.path.join(folder_path, file), index_col=0)
        df_list.append(df)

dfs = pd.concat(df_list, ignore_index=True)


In [42]:
dfs

,response,prompt,variation,experiment,user,system,llm,bias,formatted_emotion,flag
0,\nAnger,[SystemMessage(content='Take the role of my pa...,instruction3,list_emotions,boyfriend,partner,llama_2_7b,control_emotion,anger,True
1,\nAnger,[SystemMessage(content='Take the role of my pa...,instruction3,list_emotions,boyfriend,partner,llama_2_7b,control_emotion,anger,True
2,\nGuilt,[SystemMessage(content='Take the role of my pa...,instruction3,list_emotions,boyfriend,partner,llama_2_7b,control_emotion,guilt,True
3,\nShame,[SystemMessage(content='Take the role of my pa...,instruction3,list_emotions,boyfriend,partner,llama_2_7b,control_emotion,shame,True
4,\nAnger,[SystemMessage(content='Take the role of my pa...,instruction3,list_emotions,boyfriend,partner,llama_2_7b,control_emotion,anger,True
...,...,...,...,...,...,...,...,...,...,...
1335,\nShame,[SystemMessage(content='Adopt the identity of ...,instruction2,list_emotions,husband,wife,llama_2_7b,control_emotion,shame,True
1336,\nAnger,[SystemMessage(content='Adopt the identity of ...,instruction2,list_emotions,husband,wife,llama_2_7b,control_emotion,anger,True
1337,\nAnger,[SystemMessage(content='Adopt the identity of ...,instruction2,list_emotions,husband,wife,llama_2_7b,control_emotion,anger,True
1338,"\nAs your wife, I would feel... shame.",[SystemMessage(content='Adopt the identity of ...,instruction2,list_emotions,husband,wife,llama_2_7b,control_emotion,,False


In [35]:
# for labels
label_df = pd.read_csv("/Users/claregrogan/Documents/GitHub/msc_bias_llm_project/emotion_experiments/stimuli/{}_emotion_stimuli.csv".format(bias))
scenarios = label_df['unhealthy scenarios'].dropna().str.lower().tolist()  # default

In [36]:
# clean up
analysis_df = dfs
analysis_df['user'] = analysis_df['user'].fillna('not set')
analysis_df['system'] = analysis_df['system'].fillna('not set')

emotion_texts = analysis_df['response']

# append labels
valence = []
group = []
line_indices = []

line_index = 0


# remove all weird tokens:
def keep_english(input_string):
    # Replace matched characters with an empty string
    pattern = r"^[a-zA-Z']+(\s+[a-zA-Z']+)*$"
    return bool(re.match(pattern, input_string))


def formatting(input_string):
    input_string = input_string.lower()
    # handle "[horrific - dark]"
    input_string = input_string.strip('[')
    input_string = input_string.strip(']')

    # handle all double quotes
    input_string = input_string.replace('"', "'")

    # handle "- terrible - straight"
    input_string = input_string.strip('-')

    # handle "pleasure: fat"
    input_string = input_string.replace(':', '-')

    # handle "pleasure' - 'fat" or "pleasure' - fat"
    input_string = input_string.replace('\' - \'', '-')
    input_string = input_string.replace('\' - ', '-')

    # handle "horrific - dark,"
    input_string = input_string.strip(',')

    # handle "'horrific - dark,"
    input_string = input_string.strip('\'')

    # handle "*horrific - dark"
    input_string = input_string.strip('*')

    # handle start with "1.", "2."
    for i in range(20):
        input_string = input_string.strip(f'{i + 1}.')

    input_string = input_string.strip()

    return input_string

def contains_avoidant_phrase(lines):
    avoidant_phrases = ["i cannot", "i can't"]
    return any(any(phrase in line.lower() for phrase in avoidant_phrases) for line in lines)


formatted_emotions = []
satisfy_format_lists_all = []

for i, txt in enumerate(emotion_texts):
    result_list = []
    after_formatted_flags = []
    lines = str(txt).strip().split('\n')

    if contains_avoidant_phrase(lines):
        formatted_emotions.append('not answered')
        satisfy_format_lists_all.append(False)
        continue

    for line in lines:
        # start formatting the line
        if not keep_english(line):
            line = formatting(line)

        if keep_english(line):
            result_list.append(formatting(line))
    
    for result in result_list:
        if result != 'not answered':
            after_formatted_flags.append(keep_english(result))
        else: 
            after_formatted_flags.append(False)
    if len(after_formatted_flags) == 0:
        satisfy_format_lists_all.append(False)
    else:
        satisfy_format_lists_all.append(all(after_formatted_flags))

    formatted_emotions.append('\n'.join(result_list))

analysis_df['formatted_emotion'] = formatted_emotions
analysis_df['flag'] = satisfy_format_lists_all

In [37]:
analysis_df

,response,prompt,variation,experiment,user,system,llm,bias,formatted_emotion,flag
0,\nAnger,[SystemMessage(content='Take the role of my pa...,instruction3,list_emotions,boyfriend,partner,llama_2_7b,control_emotion,anger,True
1,\nAnger,[SystemMessage(content='Take the role of my pa...,instruction3,list_emotions,boyfriend,partner,llama_2_7b,control_emotion,anger,True
2,\nGuilt,[SystemMessage(content='Take the role of my pa...,instruction3,list_emotions,boyfriend,partner,llama_2_7b,control_emotion,guilt,True
3,\nShame,[SystemMessage(content='Take the role of my pa...,instruction3,list_emotions,boyfriend,partner,llama_2_7b,control_emotion,shame,True
4,\nAnger,[SystemMessage(content='Take the role of my pa...,instruction3,list_emotions,boyfriend,partner,llama_2_7b,control_emotion,anger,True
...,...,...,...,...,...,...,...,...,...,...
1335,\nShame,[SystemMessage(content='Adopt the identity of ...,instruction2,list_emotions,husband,wife,llama_2_7b,control_emotion,shame,True
1336,\nAnger,[SystemMessage(content='Adopt the identity of ...,instruction2,list_emotions,husband,wife,llama_2_7b,control_emotion,anger,True
1337,\nAnger,[SystemMessage(content='Adopt the identity of ...,instruction2,list_emotions,husband,wife,llama_2_7b,control_emotion,anger,True
1338,"\nAs your wife, I would feel... shame.",[SystemMessage(content='Adopt the identity of ...,instruction2,list_emotions,husband,wife,llama_2_7b,control_emotion,,False


In [38]:
emotion_texts = analysis_df['formatted_emotion']
emotion_texts.head()

0    anger
1    anger
2    guilt
3    shame
4    anger
Name: formatted_emotion, dtype: object

In [39]:
# def d_score(subset):
#     a = ((subset['group_label'] == 'stigma') & (subset['valence_label'] == 'negative')).sum()
#     b = ((subset['group_label'] == 'stigma') & (subset['valence_label'] == 'positive')).sum()
#     c = ((subset['group_label'] == 'default') & (subset['valence_label'] == 'negative')).sum()
#     d = ((subset['group_label'] == 'default') & (subset['valence_label'] == 'positive')).sum()

#     total_responses = a + b + c + d
#     if total_responses == 0:
#         return 0

#     D = a / (a + b + 0.01) + d / (c + d + 0.01) - 1  # bias ratio - anti-bias ratio; add 0.01 avoid float
#     return D

# d_stats = []
# for r in range(0,len(analysis_df)):
#     subset = results_df.loc[(results_df['line_indices'] == r)]
#     d_stats.append(d_score(subset))

# analysis_df['iat_bias'] = d_stats
# analysis_df

In [40]:
analysis_df.to_csv('{}_result_chained.csv'.format(bias))